### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import math
from keras import callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.utils import plot_model

### Importing Data

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv("/content/drive/MyDrive/test.csv")
id = test['id']
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])
train.head()

,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,20,50,0.000000,0.083334,0,5.837492
1,1,20,50,0.033652,18.383041,0,5.907794
2,1,20,50,0.067514,22.509278,0,7.876254
3,1,20,50,0.101542,22.808822,0,11.742872
4,1,20,50,0.135756,25.355850,0,12.234987


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Checking for Features Datatype

In [ ]:
def check(df):
    categorical_data = []
    numerical_data = []
    for i in df.columns:
        if df[i].value_counts().count() > 10:
            numerical_data.append(i)
        else:
            categorical_data.append(i)
    return([numerical_data,categorical_data])

In [ ]:
t = check(train)
print("Numerical columns: ",t[0])
print("Categorical columns: ",t[1])

Numerical columns:  ['breath_id', 'time_step', 'u_in', 'pressure']
Categorical columns:  ['R', 'C', 'u_out']


### Removing Pressure Feature's Outliers

In [ ]:
train['pressure'] = np.where(train['pressure'] < 0, np.nan, train['pressure'])
train = train.dropna()
train.isnull().sum()

breath_id    0
R            0
C            0
time_step    0
u_in         0
u_out        0
pressure     0
dtype: int64

In [ ]:
x_train = train.iloc[:,0:6]
y_train = train.iloc[:,6]

### Editing Time Step

In [ ]:
x_train['time_step'] = pd.cut(x=x_train['time_step'], bins=[-0.99, 0.5, 1, 1.5, 2, 2.5, 3], labels=[0,1,2,3,4,5])
x_train['time_step'] = x_train['time_step'].astype('int64')
x_train.head()

,breath_id,R,C,time_step,u_in,u_out
0,1,20,50,0,0.083334,0
1,1,20,50,0,18.383041,0
2,1,20,50,0,22.509278,0
3,1,20,50,0,22.808822,0
4,1,20,50,0,25.355850,0


In [ ]:
test['time_step'] = pd.cut(x=test['time_step'], bins=[-0.99, 0.5, 1, 1.5, 2, 2.5, 3], labels=[0,1,2,3,4,5])
test['time_step'] = test['time_step'].astype('int64')
test.head()

,breath_id,R,C,time_step,u_in,u_out
0,0,5,20,0,0.000000,0
1,0,5,20,0,7.515046,0
2,0,5,20,0,14.651675,0
3,0,5,20,0,21.230610,0
4,0,5,20,0,26.320956,0


### Adding Extra Features

In [ ]:
def create_new_feat(df):
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    #print("Step-1...Completed")

    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    #print("Step-2...Completed")

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    #print("Step-3...Completed")

    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    #print("Step-4...Completed")

    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']

    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    #print("Step-5...Completed")

    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = (df\
                           .groupby('breath_id')['u_in']\
                           .ewm(halflife=9)\
                           .mean()\
                           .reset_index(level=0,drop=True))
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"})\
                                                               .reset_index(level=0,drop=True))
    #print("Step-6...Completed")

    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    #print("Step-7...Completed")

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    #print("Step-8...Completed")

    return df

In [ ]:
x_train = create_new_feat(x_train)
#x_train = x_train.fillna(x_train.min())

test = create_new_feat(test)
#test = test.fillna(test.min())

In [ ]:
t = check(x_train)
print("Numerical columns: ",t[0])
print("\nCategorical columns: ",t[1])

In [ ]:
x_train = x_train.drop(['breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'],axis=1)

test = test.drop(['breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'],axis=1)

print(f"Train: {x_train.shape} \nTest: {test.shape}")

### Normalize the Data using Min Max Scalar

In [ ]:
scaler1 = RobustScaler()
scaler2 = RobustScaler()
scaler3 = RobustScaler()

x_train = scaler1.fit_transform(x_train)
test = scaler3.fit_transform(test)
y_train = scaler2.fit_transform(y_train.values.reshape(-1,1))

In [ ]:
x_train = x_train.reshape(-1,1,64)
test = test.reshape(-1,1,64)
y_train = y_train.reshape(-1,1,1)

### Create and fit the LSTM network

In [ ]:
model = Sequential()
model.add(LSTM(150,input_shape=(1,64),return_sequences=True))
model.add(LSTM(100,return_sequences=True,input_shape=(1,64)))
model.add(LSTM(64,return_sequences=True))
model.add(Dense(16,activation='selu'))
model.add(Dense(1))
model.summary()

model.compile(loss='mae',optimizer='adam')

In [ ]:
#epoch=300
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",mode ="min",patience = 3,restore_best_weights = True)
model.fit(x_train,y_train,epochs = 50,batch_size = 512,callbacks =[earlystopping])

In [ ]:
y_test = model.predict(test)

In [ ]:
test_y = scaler2.inverse_transform(y_test.reshape(-1,1))
y_test.shape

In [ ]:
y_train.shape

In [ ]:
plot_model(
    model,
    to_file='Model.png',
    show_shapes=True,
    show_layer_names=True
)

In [ ]:
prediction = pd.concat([id,pd.DataFrame(test_y)],axis=1)
prediction.columns=['id','pressure']
prediction.to_csv("Keras_Ann.csv",index=False)

In [ ]:
prediction.head()